In [4]:
import os
import glob
import random
from sklearn.model_selection import train_test_split

In [5]:
os.system("mkdir ../data/data_splits")
smiles_cluster_data = glob.glob("../data/preprocessed_data/smiles_cluster*.txt")
num_clusters = 10 # Number of clusters to choose to do CV on. The entire data would take longer to do CV. A subset of data is chosen for CV

In [6]:
for files in smiles_cluster_data:
    # Reading data
    distance_threshold = files.split("_")[-1][:-4]
    smiles_cluster = {entry.split()[0]:int(entry.split()[1]) for entry in open(files,"r").readlines()}
    all_smiles = list(smiles_cluster.keys())
    all_cluster = list(smiles_cluster.values())
    
    # Seleting random clusters for subset of data
    chosen_random_clusters = [random.randint(0, len(set(all_cluster))) for _ in range(num_clusters)]
    
    # get subset of data using chosen random clusters
    subset_smiles_cluster = {smiles:smiles_cluster[smiles] for smiles in smiles_cluster if smiles_cluster[smiles] in chosen_random_clusters}
    
    
    # Renaming the clusters in the subset, needed for deeplearning. naming should start from 0-n
    ref_name = {list(set(subset_smiles_cluster.values()))[i]:i for i in range(len(set(subset_smiles_cluster.values())))}
    renamed_smiles_cluster = {}
    for smiles in subset_smiles_cluster:
        renamed_smiles_cluster[smiles] = ref_name[subset_smiles_cluster[smiles]]
    
    
    # Divide the data to train,valid and test for final model
    X_train, X_valid_test, y_train, y_valid_test = train_test_split(all_smiles, all_cluster, test_size=0.2, random_state=0,stratify=all_cluster)
    X_valid, X_test, y_valid, y_test = train_test_split(X_valid_test, y_valid_test, test_size=0.5, random_state=0,stratify=y_valid_test)
    
    # Saving outputs
    with open("../data/data_splits/train_" + str(distance_threshold) + ".txt","w") as f:
        for x,y in zip(X_train,y_train):
            f.write(x + " " + str(y) + "\n")
            
    with open("../data/data_splits/valid_" + str(distance_threshold) + ".txt","w") as f:
        for x,y in zip(X_valid,y_valid):
            f.write(x + " " + str(y) + "\n")
            
    with open("../data/data_splits/test_" + str(distance_threshold) + ".txt","w") as f:
        for x,y in zip(X_test,y_test):
            f.write(x + " " + str(y) + "\n")
            
            
    # Divide the data to train and test for CV
    subset_smiles = list(renamed_smiles_cluster.keys())
    subset_cluster = list(renamed_smiles_cluster.values())
    X_train, X_test, y_train, y_test = train_test_split(subset_smiles, subset_cluster, test_size=0.1, random_state=0,stratify=subset_cluster)
    
    # Saving outputs
    with open("../data/data_splits/CV_train_" + str(distance_threshold) + ".txt","w") as f:
        for x,y in zip(X_train,y_train):
            f.write(x + " " + str(y) + "\n")
    
    with open("../data/data_splits/CV_test_" + str(distance_threshold) + ".txt","w") as f:
        for x,y in zip(X_test,y_test):
            f.write(x + " " + str(y) + "\n")